### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
# School ID | school_name | type | size | budget
school_df = pd.read_csv(school_data_to_load)
# Student ID | student_name | gender | grade | school_name | reading_score | math_score
student_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
merged_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

district_summary_df = None

def get_district_summary() -> pd.DataFrame:
    global district_summary_df
    if district_summary_df is None:
        school_count = school_df["School ID"].nunique()
        student_count = student_df["Student ID"].nunique()
        total_budget = school_df["budget"].sum()
        math_average = student_df["math_score"].mean()
        reading_average = student_df["reading_score"].mean()
        math_passing_count = sum(student_df["math_score"] >= 70)/student_count
        reading_passing_count = sum(student_df["reading_score"] >= 70)/student_count
        passing_count = sum((student_df["reading_score"] >= 70) & (student_df["math_score"] >= 70))/student_count
        district_summary_df = pd.DataFrame(data = [[school_count, student_count, total_budget, math_average, reading_average, math_passing_count, reading_passing_count, passing_count]], index = ["Summary"], columns = ["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"])
    return district_summary_df

def district_summary() -> pd.DataFrame:
    return get_district_summary().style.format({
        'Total Students': '{:,}'.format,
        'Total Budget': '${:,.2f}'.format,
        'Average Math Score': '{:,.2f}%'.format,
        'Average Reading Score': '{:,.2f}%'.format,
        '% Passing Math': '{:,.2%}'.format,
        '% Passing Reading': '{:,.2%}'.format,
        '% Passing Overall': '{:,.2%}'.format
})

school_summary_df = None

def get_school_summary() -> pd.DataFrame:
    global school_summary_df
    if school_summary_df is None:
        total_students_s = merged_df.groupby("School ID")["Student ID"].nunique().rename("Total Students")
        school_data_s = school_df.set_index("School ID").loc[:, ["school_name", "type", "budget"]].rename(columns = {"school_name" : "School Name", "type" : "School Type", "budget" : "Total School Budget"})
        per_student_budget_s = (school_data_s["Total School Budget"]/total_students_s).rename("Per Student Budget")
        ave_math_reading_s = merged_df.groupby("School ID")[["math_score", "reading_score"]].mean().rename(columns = {"math_score" : "Average Math Score", "reading_score" : "Average Reading Score"})
        students_passing_math_reading_s = (merged_df.set_index("School ID")[["math_score", "reading_score"]] >= 70)
        students_passing_both_s = (students_passing_math_reading_s["math_score"] & students_passing_math_reading_s["reading_score"])
        passing_math_reading_count_s = students_passing_math_reading_s.groupby("School ID").sum().rename(columns = {"math_score" : "Students Passing Math", "reading_score" : "Students Passing Reading"})
        passing_math_reading_percent_s = passing_math_reading_count_s.div(total_students_s, axis = 0).rename(columns = {"Students Passing Math" : "% Passing Math", "Students Passing Reading" : "% Passing Reading"})
        passing_both_count_s = students_passing_both_s.groupby("School ID").sum()
        passing_both_percent_s = (passing_both_count_s/total_students_s).rename("% Passing Overall")

        school_summary_df = pd.DataFrame(pd.concat([total_students_s, school_data_s, per_student_budget_s, ave_math_reading_s, passing_math_reading_percent_s, passing_both_percent_s], axis = 1), columns = ["School Name", "School Type", "Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"])
    return school_summary_df

def school_summary(head : int = 0):
    if (head > 0): return format_school_summary(get_school_summary().set_index("School Name").sort_index().head(head))
    else: return format_school_summary(get_school_summary().set_index("School Name").sort_index())

def format_school_summary(df : pd.DataFrame):
    return df.style.format({
    'Total Students': '{:,}'.format,
    'Total School Budget': '${:,.2f}'.format,
    'Per Student Budget': '${:,.2f}'.format,
    'Average Math Score': '{:,.2f}%'.format,
    'Average Reading Score': '{:,.2f}%'.format,
    '% Passing Math': '{:,.2%}'.format,
    '% Passing Reading': '{:,.2%}'.format,
    '% Passing Overall': '{:,.2%}'.format
})

def top_schools(head : int = 5):
    return format_school_summary(get_school_summary().set_index("School Name").nlargest(head, "% Passing Overall"))
    
def bottom_schools(tail : int = 5):
    return format_school_summary(get_school_summary().set_index("School Name").nsmallest(tail, "% Passing Overall"))

school_math_scores_df = None

def get_math_scores_by_grade():
    global school_math_scores_df
    if school_math_scores_df is None:
        school_math_scores_df = pd.DataFrame(merged_df.rename(columns = {"school_name" : "School Name", "grade" : "Grade"}).groupby(["School ID", "School Name", "Grade"])["math_score"].mean()).reset_index(drop = False).pivot(index = "School Name", columns = "Grade", values = "math_score")[["9th", "10th", "11th", "12th"]]
    return school_math_scores_df

def format_scores_by_grade(df : pd.DataFrame):
    return df.style.format('{:,.2f}%')

school_reading_scores_df = None

def math_scores_by_grade():
    return format_scores_by_grade(get_math_scores_by_grade())

def get_reading_scores_by_grade():
    global school_reading_scores_df
    if school_reading_scores_df is None:
        school_reading_scores_df = pd.DataFrame(merged_df.rename(columns = {"school_name" : "School Name", "grade" : "Grade"}).groupby(["School ID", "School Name", "Grade"])["reading_score"].mean()).reset_index(drop = False).pivot(index = "School Name", columns = "Grade", values = "reading_score")[["9th", "10th", "11th", "12th"]]
    return school_reading_scores_df

def reading_scores_by_grade():
    return format_scores_by_grade(get_reading_scores_by_grade())

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


# IN PROGRESS
###########################################################################
# DONE

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
## use get_district_summary() for the raw data
# get_district_summary() 
## without get_ to get the formatter/pretty print
district_summary()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Summary,15,"39,170","$24,649,428.00",78.99%,81.88%,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
## use get_school_summary() for the raw data
# get_school_summary() 
## without get_ to get the formatter/pretty print
school_summary(1)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05%,81.03%,66.68%,81.93%,54.64%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
## use get_school_summary() for the raw data
# get_school_summary()
top_schools(1)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,91.33%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
## use get_school_summary() for the raw data
# get_school_summary()
bottom_schools(1)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84%,80.74%,66.37%,80.22%,52.99%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
math_scores_by_grade()

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
reading_scores_by_grade()

Grade,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


# DONE
###########################################################################
# TODO

## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
